# Wordle Bot
Automatyzacja gry w Wordle po angielsku (`https://wordly.org`). 

In [1]:
import selenium as se
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from pandas import DataFrame, read_csv
import numpy as np
from collections import Counter
import random

In [2]:
driver=webdriver.Firefox()
driver.get("https://wordly.org")

Pozbycie się okienka odnośnie zbierania danych

In [3]:
manage_option=driver.find_element(by=By.CLASS_NAME, value="fc-cta-manage-options")
manage_option.click()
confirm_choices=driver.find_element(by=By.CLASS_NAME, value="fc-confirm-choices")
confirm_choices.click()
#TODO:
# - dodać odrzucenie wyborów


In [6]:
def filter_words(word,info_list)->bool:
    ...

print(filter_words("diddy",[("q",0),("")]))

None


In [4]:
def is_word_correct(word:str,info_list:list[tuple[str,int]])->bool:
    for index,i in enumerate(info_list):
        if i[1]==0:
            char_num=0
            for inf in info_list:
                if inf[0]==i[0] and inf[1]!=0:
                    char_num+=1
            for char in word:
                if word.count(i[0])<=char_num:
                    if char==i[0].lower():
                        return False
                else:
                    return False
                
        elif i[1]==1:
            res=False
            for j,char in enumerate(word):
                if index==j and char==i[0].lower():
                    return False
                if index!=j and char==i[0].lower():
                    res=True
            if not res:
                return False
        else:
            if word[index]!=i[0].lower():
                return False
    return True
                
#TODO:
# uwzględnij duplikaty w literach

# Strategia początkowa - CRANE

In [5]:
turn=0
word_list=["crane"]

Wpisanie słowa

In [17]:
input_word=random.choice(word_list)
keyboard=driver.find_element(by=By.CLASS_NAME, value="Game-keyboard")
letter_buttons=keyboard.find_elements(by=By.CLASS_NAME, value="Game-keyboard-button")
enter_button=None
for button in letter_buttons:
    if button.text=="Enter":
        enter_button=button
if enter_button is None:
    raise ValueError
for letter in input_word:
    print(letter)
    for button in letter_buttons:
        if button.text.lower()==letter:
            print(button.rect)
            button.click()
enter_button.click()


r
{'x': 223.5, 'y': 416.0, 'width': 54.0, 'height': 46.0}
i
{'x': 463.5, 'y': 416.0, 'width': 54.0, 'height': 46.0}
n
{'x': 429.683349609375, 'y': 520.0, 'width': 51.44999694824219, 'height': 46.0}
s
{'x': 110.16667175292967, 'y': 468.0, 'width': 60.666656494140625, 'height': 46.0}
e
{'x': 163.5, 'y': 416.0, 'width': 54.0, 'height': 46.0}


Przechwycenie wyniku

In [12]:
placement_value={
    "letter-absent":0,
    "letter-elsewhere":1,
    "letter-correct":2
}
rows=driver.find_element(by=By.CLASS_NAME,value="game_rows").find_elements(by=By.CLASS_NAME,value="Row-locked-in")
print(len(rows))
info=[]
print(turn)
row_letters=rows[turn].find_elements(by=By.CLASS_NAME,value="Row-letter")
print(row_letters[0].get_attribute("class"))
for row_letter in row_letters:
    print(row_letter.rect)
    print(row_letter.get_attribute("class"))
    letter=row_letter.text.lower()
    placement_class=row_letter.get_attribute("class").split(" ")[1]
    info.append((letter,placement_value[placement_class]))
    print(info)
print(info)

    
    


2
1
Row-letter letter-elsewhere
{'x': 208.5, 'y': 112.0, 'width': 48.0, 'height': 48.0}
Row-letter letter-elsewhere
[('n', 1)]
{'x': 262.5, 'y': 112.0, 'width': 48.0, 'height': 48.0}
Row-letter letter-absent
[('n', 1), ('u', 0)]
{'x': 316.5, 'y': 112.0, 'width': 48.0, 'height': 48.0}
Row-letter letter-elsewhere
[('n', 1), ('u', 0), ('r', 1)]
{'x': 370.5, 'y': 112.0, 'width': 48.0, 'height': 48.0}
Row-letter letter-correct
[('n', 1), ('u', 0), ('r', 1), ('s', 2)]
{'x': 424.5, 'y': 112.0, 'width': 48.0, 'height': 48.0}
Row-letter letter-correct
[('n', 1), ('u', 0), ('r', 1), ('s', 2), ('e', 2)]
[('n', 1), ('u', 0), ('r', 1), ('s', 2), ('e', 2)]


Załadowanie listy słów

In [13]:
if turn==0:
    word_list=read_csv("./word-bank.csv") 
    word_list=word_list.values.flatten()
    print(word_list)
    print(len(word_list))
    # valid words ma więcej liter, sprawdzić czy word bank styknie 

Filtracja listy słów

In [14]:
new_list=[]
for word in word_list:
    if is_word_correct(word,info):
        new_list.append(word)
word_list=np.array(new_list)
print(new_list)
print(len(new_list))


[np.str_('rinse')]
1


Następna tura

In [15]:
turn+=1

powtarzamy aż zwyciężymy